INVESTIGATING OR ANALYSING MATERIALS BY DETERMINING THEIR CHEMICAL OR PHYSICAL PROPERTIES

I – Term detection

1-1	What do we call as a term?

It a phrase or multiword or a word that have

-	A clear and specific meaning within the context of the patent

-	It should not a common word or phrase found in everyday language

-	A technical or scientific concept, material, method or device that is specific to the invention described in the patent.

1-2	Create a gold dataset

1-3	Build a rule-based baseline (may using re or generating n-grams and filter it)

1-4	Train a statistical model (bases on spacy)

1-5	Improve annotation using prodigy

1-6	Evaluate the model 

1-7	Iterate and refine the rule-based line


In [2]:
import sys
import time
import json
import pprint
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy.matcher import PhraseMatcher, Matcher
from spacy.util import filter_spans
from spacy import displacy
from spacy.tokens import DocBin
from spacy.tokens import Span
from collections import Counter
from nltk.tokenize import MWETokenizer
from nltk.util import Trie
import nltk
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('omw-1.4')
tqdm.pandas()
spacy.__version__


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\etien\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\etien\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


'3.3.2'

In [3]:
spacy.require_gpu()

True

In [4]:
patent_data = open('G01N.txt').read().strip()

# split into patents texts | 1 entry = 1 patent
patent_texts = patent_data.split('\n\n')

# split each patent into lines
patent_lines = patent_data.split('\n')

In [5]:
len(patent_lines)

330594

In [6]:
len(patent_texts)

2139

In [7]:
def get_feature_names(vectorizer):
    if hasattr(vectorizer, 'get_feature_names'):
        return vectorizer.get_feature_names()
    else:
        return vectorizer.get_feature_names_out()

In [8]:
# here are the potential terms
mwes = open('manyterms.lower.txt').read().lower().strip().split('\n')
print(mwes[44444:44456])
print(len(mwes), 'mwes')

['antonio superchi', 'antonio tarver', 'antonio torres jurado', 'antonio valdes', 'antonio valdes y fernandez bazan', 'antonio valdez', 'antonio valdés y bazán', 'antonio valdés y fernández bazán', 'antonio valente', 'antonio vitali', 'antonio vivaldi', 'antonio xavier machado e cerveira']
743274 mwes


In [9]:
# Here lowercase=False option is used to keep the original case of the terms, since we possibly could have term abbreviations. Like API, CAT, etc.
cvectorizer = CountVectorizer(ngram_range=(
    1, 4), stop_words="english", vocabulary=mwes, lowercase=True)
X = cvectorizer.fit_transform(patent_texts)

# Show top-25 most frequent terms
termdf_cv = pd.DataFrame(np.sum(X, axis=0), columns=get_feature_names(cvectorizer)).T.sort_values(by=0, ascending=False)
termdf_cv.head(25)

c:\Users\etien\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


,0
amino acid,24339
nucleic acid,13185
amino acid sequence,11407
light source,6943
amino acids,6170
control unit,5249
nucleotide sequence,4374
variable region,3264
mass spectrometry,3091
monoclonal antibody,2917


In [9]:
termdf_cv = termdf_cv[termdf_cv[0] >= 1]
termdf_cv.to_csv('terms.tsv', sep='\t')

In [10]:
# !python -m spacy download en_core_web_lg
#!python -m spacy download en_core_web_trf

In [22]:
nlp = spacy.load("en_core_web_lg")
shortText = patent_texts[0]
doc = nlp(shortText)
#displacy.render(doc, style="ent", jupyter=True)

In [23]:
# noun_chunks = [{
#     'start': chunk.start_char,
#     'end': chunk.end_char,
#     'label': 'NC',
#     'text': chunk.text
# } 
#     for chunk in doc.noun_chunks
# ]
# funkycolor = "linear-gradient(90deg, #aa9cfc, #fc9ce7)"
# displacy.render({'text': shortText, 'ents': noun_chunks, 'title': None},
#                 style='ent', options={'colors': {"NC": funkycolor}}, manual=True, jupyter=True)


In [24]:
import re


def combineFreqDics(freq_dic):
    """
    function that combines entries of the freq_dic if they only differ in case into the most frequent case version
    """
    combined_dict = {}

    for key, value in freq_dic.items():
        normalized_key = key.lower()
        if normalized_key not in combined_dict:
            combined_dict[normalized_key] = (key, value)
        else:
            orig_key, orig_value = combined_dict[normalized_key]
            combined_dict[normalized_key] = (
                key if value > orig_value else orig_key, orig_value + value)

    return {key: value for key, value in combined_dict.values()}


def is_plural(singular, plural):
    return singular + 's' == plural or singular == plural + 's'


def rawExtractRefs(text, allnums=True, multipleResults=True):
    """
    text: the text potentially containing reference numerals
    NO: termjson: json as send back to the frontend
    allnums: if true also terms without parentheses
    """
    if allnums:
        renum = re.compile(r'[ (]\d+[) .,]')
    else:
        renum = re.compile(r'\(\d+\)')
    num2term = {}
    # print(111,'a\ba',222,r'\b|\b'.join('the a an to for under behind above least on which and or Figure \W*\d+\W*'.split()),333,'\b'+'\b|\b'.join('the a an to for under behind above least on which and or Figure \W*\d+\W*'.split())+r'\b')
    excluders = '''the a an 
    to for of under behind above least on which and or more all 
    Claim Claims Figure Figures Fig Figs'''
    noterms = re.compile(r'\b'+r'\b|\b'.join((excluders +
                         ' \W*\d+\W* \W*[,."()]+\W*').split())+r'\b'+r'|\W*[:;,?*%$=+()]+\W*', re.I)
    for m in renum.finditer(text):  # all number mathes
        i = int(m.group(0)[1:-1])

        potwords = text[0:m.start(0)].split()[-4:]
        # print(222,potwords)
        for j in range(len(potwords)):
            tu = ' '.join(potwords[j:])
            if not noterms.search(tu):
                num2term[i] = num2term.get(i, {})
                num2term[i][tu] = num2term[i].get(tu, 0)+1
        # print(333, [tuple(potwords[j:]) for j in range(len(potwords))], i)
    # print(444, num2term)
    num2termlist = {}
    for num, td in num2term.items():
        ctd = combineFreqDics(td)
        # print("555 ctd:",ctd)
        ma = max(ctd.values())
        mas = [(k, v) for k, v in ctd.items() if v == ma]
        # print('mas',mas)
        filtered_list = []
        for t, j in mas:
            add_to_list = True
            for s, i in mas:
                # is subset the right thing here?
                if t != s and set(t).issubset(set(s)):
                    add_to_list = False
                    break
            if add_to_list:
                filtered_list.append(t)
                # Check for less frequent terms that just differ in number
                for key, value in ctd.items():
                    if is_plural(t, key) and key not in filtered_list:
                        filtered_list.append(key)
                    elif is_plural(key, t) and t not in filtered_list:
                        filtered_list.append(t)

        # print('--->',filtered_list)
        num2termlist[num] = filtered_list
    return num2termlist


num2termlist = rawExtractRefs(shortText)
display(num2termlist)

def removeNumbers(text, num2termlist):
    terms = [val for sublist in num2termlist.values() for val in sublist]
    numterms = re.compile(r'(\b'+r'\b|\b'.join(terms) +
                          r'\b) *[ (]\d+([) ]|([.,:]*)) *()', re.I)
    clean_text = numterms.sub(r'\1\3 ',text)
    return clean_text


cleanNshort = removeNumbers(shortText, num2termlist)
print(cleanNshort)


{1: ['at'],
 0: ['preferably'],
 5: ['preferably'],
 10: ['preferably'],
 6: ['item'],
 8: ['detection device'],
 101265329: ['CN'],
 20: ['preferably'],
 30: ['preferably'],
 40: ['preferably'],
 2014044184: ['Patent Application Publication WO'],
 2: ['Example'],
 104098677: ['CN'],
 100: ['chip at'],
 50: ['chip were washed with'],
 3: ['washing solution']}

SOLID-PHASE CARRIER CAPABLE OF IMPROVING DETECTION SENSITIVITY, AND DETECTION COMPONENT
_____2019_____3502257_____490788587_____EP3500000.txt_____G01N_____C08G77/38:C08K2003/2241:C08K2201/005:C08K3/22:C12N15/63:G01N33/545
Provided is a solid phase carrier capable of improving detection sensitivity, and a detection device. The solid phase carrier is provided with a polydimethylsiloxane layer with an initiator on the surface, and titanium dioxide particles distributed in said polydimethylsiloxane layer with an initiator on the surface.
_____d:
Technical Field
The present invention belongs to the field of functional materials, and in particular relates to a polydimethylsiloxane solid phase carrier with an initiator on the surface, which is capable of improving detection sensitivity, and a detection device using the solid phase carrier as a substrate.
Background Art
A polydimethylsiloxane (abbreviated as PDMS hereinafter) has excellent properties such as non-toxicity, transparency, elastic

In [25]:
def get_noun_chunks(doc):
    noun_chunks = []
    for chunk in doc.noun_chunks:
        # Remove determiners from the noun chunk
        chunk_text = ' '.join([token.text for i, token in enumerate(
            chunk) if not (i == 0 and token.pos_ == 'DET')])

        # Get the start and end character positions of the modified noun chunk
        start_char = doc[chunk.start].idx if doc[chunk.start].pos_ != 'DET' else doc[chunk.start].nbor(
        ).idx
        end_char = start_char + len(chunk_text)

        noun_chunks.append({
            'start': start_char,
            'end': end_char,
            'label': 'NC',
            'text': chunk_text
        })
    return noun_chunks


noun_chunks = get_noun_chunks(doc)

displacy.render({'text': cleanNshort, 'ents': noun_chunks, 'title': None},
                style='ent', manual=True, jupyter=True)


In [ ]:
def get_noun_chunks(doc, remove_list=[], exclude_list=[]):
    noun_chunks = []
    for chunk in doc.noun_chunks:
        exclude_list += ['that', 'which']
        if any(token.text in exclude_list for token in chunk):
            continue

        filtered_tokens = [
            token for token in chunk
            if token.dep_ != "relcl" and token.text not in remove_list and token.pos_ != 'DET' and not token.is_punct
        ]

        if not filtered_tokens:
            continue

        chunk_text = ' '.join(token.text for token in filtered_tokens)
        chunk_text = chunk_text.replace('"', '').replace("'", "")

        start_char = filtered_tokens[0].idx
        end_char = start_char + len(chunk_text)

        noun_chunks.append({
            'start': start_char,
            'end': end_char,
            'label': 'NC',
            'text': chunk_text
        })

    return noun_chunks


remove_list = ['said', 'simply']
exclude_list = ['FIG', 'FIGS']

noun_chunks = get_noun_chunks(doc, remove_list, exclude_list)

displacy.render({'text': cleanNshort, 'ents': noun_chunks, 'title': None},
                style='ent', manual=True, jupyter=True)




In [12]:
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(text) for text in termdf_cv.index]
matcher.add("CH", patterns)

In [13]:
train_lines, test_lines = train_test_split(
    patent_lines, test_size=0.3, random_state=42)


print(len(train_lines))
print(len(test_lines))

231415
99179


In [14]:
def create_dataset(text, n_lines, filename, offset=0):
    LABEL = "CH"
    doc_bin = DocBin()  # create a DocBin object

    for training_example in tqdm(text[offset:offset+n_lines]):
        doc = nlp.make_doc(training_example)
        ents = []

        for match_id, start, end in matcher(doc):
            span = Span(doc, start, end, label=LABEL)
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)

        filtered_ents = filter_spans(ents)
        doc.ents = filtered_ents
        doc_bin.add(doc)
    doc_bin.to_disk(filename)

In [16]:
create_dataset(train_lines, 60000, "training_data.spacy")
create_dataset(test_lines, 2000, "test_data.spacy")
create_dataset(test_lines, 3000, "valid_data.spacy", offset=2000)

  0%|          | 0/60000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

In [17]:
# save train_lines to txt file
with open('train_lines.txt', 'w') as f:
    for line in train_lines:
        f.write(line)
        f.write('\n')
f.close()

# save train_lines to txt file
with open('valid_lines.txt', 'w') as f:
    for line in test_lines[2000:5000]:
        f.write(line)
        f.write('\n')
f.close()

# save test_lines to txt file
with open('test_lines.txt', 'w') as f:
    for line in test_lines[0:2000]:
        f.write(line)
        f.write('\n')
f.close()

In [18]:
# Run to generate full training config
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [19]:
!python -m spacy train config.cfg --output ./spacy_output --paths.train ./training_data.spacy --paths.dev ./valid_data.spacy

✔ Created output directory: spacy_output

[2023-04-14 13:15:38,664] [INFO] Set up nlp object from config
[2023-04-14 13:15:38,674] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-14 13:15:38,678] [INFO] Created vocabulary
[2023-04-14 13:15:38,681] [INFO] Finished initializing nlp object
[2023-04-14 13:16:46,998] [INFO] Initialized pipeline components: ['tok2vec', 'ner']



ℹ Saving to output directory: spacy_output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     49.50    2.62    1.63    6.67    0.03
  0     200         31.07   2389.20   16.91   65.13    9.72    0.17
  0     400         31.20    836.19   31.39   60.78   21.16    0.31
  0     600         51.40   1028.38   42.12   55.07   34.11    0.42
  0     800       2755.18   1037.90   21.01   85.20   11.98    0.21
  0    1000       9404.46    939.89   45.81   59.92   37.08    0.46
  0    1200        137.78   1124.66   19.47   48.13   12.21    0.19
  0    1400        204.03   1574.53   50.91

In [20]:
# Load best model
nlp= spacy.load("./spacy_output/model-best")

# Just text snippet
doc = nlp("""The scientific and technological terminologies referred to herein have the same meanings as what are generally understood by a person skilled in the art, and if there is a conflict, the definition in the present description shall prevail.
Firstly, in one aspect, the present invention provides a solid phase carrier (the solid phase carrier of the present invention) comprising:
a polydimethylsiloxane layer with an initiator on the surface, andtitanium dioxide particles distributed in said polydimethylsiloxane layer with an initiator on the surface.
Said polydimethylsiloxane with an initiator on the surface (iPDMS) belongs to the prior art, and reference can be made to Chinese Patent Publication No. CN 101265329 A.
Titanium dioxide, commonly known as titanium white, is usually a white powder. The crystal form of said titanium oxide is not particularly limited and may be, for example, of rutile type, anatase type or nanoscale ultrafine titanium dioxide.
Preferably, said titanium dioxide particles have an average particle size of 1 nm to 1000 nm, more preferably 5 nm to 500 nm, more preferably 5 nm to 200 nm, more preferably 10 nm to 100 nm, and more preferably 10 nm to 50 nm.
The specific surface area ofsaid titanium dioxide particles is preferably 10 to 500 m2/g, more preferably 20 to 400 m2/g, more preferably 30 to 300 m2/g, and more preferably 40 to 200 m2/g.
The content of said titanium dioxide particles is 0.0001 to 100 parts by weight, preferably 0.0002 to 90 parts by weight, more preferably 0.0005 to 80 parts by weight, more preferably 0.001 to 70 parts by weight, more preferably 0.002 to 60 parts by weight, more preferably 0.005 to 40 parts by weight, more preferably 0.01 to 40 parts by weight, more preferably 0.02 to 30 parts by weight, more preferably 0.05 to 20 parts by weight, and more preferably 0.1 to 10 parts by weight, relative to 100 parts by weight of said polydimethylsiloxane layer with an initiator on the surface.
Said solid phase carrier may be prepared, for example, by mixing macromolecular precursor A, cross-linking agent B, vinyl-endcapped initiator C, and said titanium dioxide particles D at a certain weight ratio, and leaving the mixture to stand, for example, for 6 to 24 hours to form an elastomer. In the mixture, said macromolecular precursor A is poly(dimethyl-methylvinylsiloxane); said cross-linking agent B is vinyl-endcapped poly(dimethyl-methylvinylsiloxane) and poly(dimethyl-methylhydrogenosiloxane); and said vinyl-endcapped initiator C is 10-undecenyl 2-bromo-2-methylpropionate.
The shape of said solid phase carrier includes, but is not limited to: beads, magnetic beads, films, microtubes, filter membranes, plates, microplates, carbon nanotubes, sensor chips, etc. Pits, grooves, filter membrane bottoms and the like may be provided on a flat solid phase carrier such as a film or a plate.
Preferably, said solid phase carrier further comprises: an oligomeric ethylene glycol methacrylate layer located on said polydimethylsiloxane layer with an initiator on the surface. Said oligomeric ethylene glycol methacrylate layer may be formed by initiating the polymerization of oligomeric ethylene glycol methacrylate on said polydimethylsiloxane layer with an initiator on the surface. Such a solid phase carrier can completely prevent non-specific adsorption of proteins.
In another aspect, the present invention provides a detection device (the detection device of the present invention) comprising: the above-mentioned solid phase carrier of the present invention, and a polypeptide or protein linked to said oligomeric ethylene glycol methacrylate layer. The manner by which said polypeptide or protein is linked to said oligomeric ethylene glycol methacrylate layer may be a covalent linkage, and reference can be made to, for example, the description of International Patent Application Publication WO 2014044184 for details. The polypeptide layer or protein herein refers to a compound obtained by the dehydration condensation of amino acid molecules, usually, the compound consisting of 2 to 50 amino acid residues is referred to as a polypeptide, and the compound consisting of 50 or more amino acid residues is referred to as a protein.
The detection device of the present invention has high detection sensitivity and can completely inhibit non-specific binding of proteins, and is thus particularly suitable for detecting a substance (e.g., a protein, a polypeptide, a small molecule compound, a nucleic acid and the like) capable of binding to the polypeptide or protein in said polypeptide layer or protein layer.
In another aspect, the present invention provides a detection kit (the detection kit of the present invention) comprising the above-mentioned detection device of the present invention or the solid phase carrier of the present invention. The detection kit of the present invention may further comprise other components, and reference can be made to, for example, the description of International Patent Application Publication WO 2014044184.
The detection kit of the present invention is also particularly suitable for detecting a substance (e.g., a protein, a polypeptide, a small molecule compound, a nucleic acid and the like) capable of binding to the polypeptide or protein in said polypeptide layer or protein layer.
Examples
The present invention will be described below in more detail by means of examples, but the present invention is not limited to these examples.
1. Preparation and confirmation of polypeptides
A peptide composed of 30 amino acids used in the examples has an amino acid sequence as set forth in SEQ ID NO: 1, which was synthesized by GL Biochem (Shanghai) Ltd.
SEQ ID NO: 1: PLVEDGVKQCDRYWPDEGASLYHVYEVNLV is positive for sera of patients with type 1 diabetes, and negative for sera of normal healthy individuals or non-type 1 diabetic patients, and reference can be made to Chinese Patent Application Publication No. CN104098677A.
2. Preparation of detection device
Detection devices 1-4 comprising SJ-modified silica gel thin films 1-4 of the invention were obtained as described in Example 2 of Chinese Patent Application Publication No. CN 104098677 A, except that in order to prepare the SJ-modified silica gel thin films 1-4 of the invention, the polydimethylsiloxane precursor A, cross-linking agent B, vinyl-endcapped initiator C and titanium oxide particles D (with an average particle size of 40 nm, a specific surface area of 60 m2/g, of rutile type) were sufficiently mixed at a ratio of A : B : C : D = 10 : 1 : 0.2 : (5, 1, 0.5 or 0.1), respectively.
Detection devices 5-8 comprising SJ-modified silica gel thin films 5-8 of the invention were obtained as described in Example 2 of Chinese Patent Application Publication No. CN 104098677 A, except that in order to prepare the SJ-modified silica gel thin films 5-8 of the invention, the polydimethylsiloxane precursor A, cross-linking agent B, vinyl-endcapped initiator C and titanium oxide particles D (with an average particle size of 20 nm, a specific surface area of 120 m2/g, of rutile type) were sufficiently mixed at a ratio of A : B : C : D = 10 : 1 : 0.2 : (5, 1, 0.5 or 0.1), respectively.
In addition, a detection device of Example 2 of Chinese Patent Application Publication No. CN 104098677 A (the SJ-modified silica gel thin film used does not comprise titanium oxide particles) was prepared as a control.
3. Detection with detection devices""")

# Show NER results
spacy.displacy.render(doc, style="ent", jupyter=True)

### Improve the model using prodigy

In [21]:
# !pip install C:\Users\etien\Downloads\prodigy-1.11.11-windows\windows\prodigy-1.11.11-cp39-cp39-win_amd64.whl

In [22]:
# !python -m prodigy

In [30]:
!python -m prodigy ner.correct gold_tech  ./spacy_output/model-best  valid_lines.txt --label CH

Using 1 label(s): CH
Added dataset val_fine_tune to database SQLite.
⚠ The model you're using isn't setting sentence boundaries (e.g. via the parser
or sentencizer). This means that incoming examples won't be split into
sentences.

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!



ERROR:    [Errno 10048] error while attempting to bind on address ('::1', 8080, 0, 0): only one usage of each socket address (protocol/network address/port) is normally permitted


In [32]:
!netstat -ano | findstr :8080

  TCP    127.0.0.1:8080         0.0.0.0:0              LISTENING       7812
  TCP    [::1]:8080             [::]:0                 LISTENING       7812


In [33]:
!Taskkill /PID <PIDProdigy> /F

The system cannot find the file specified.
